In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/food-com-recipes-and-user-interactions/RAW_interactions.csv
/kaggle/input/food-com-recipes-and-user-interactions/ingr_map.pkl
/kaggle/input/food-com-recipes-and-user-interactions/PP_recipes.csv
/kaggle/input/food-com-recipes-and-user-interactions/RAW_recipes.csv
/kaggle/input/food-com-recipes-and-user-interactions/interactions_train.csv
/kaggle/input/food-com-recipes-and-user-interactions/interactions_test.csv
/kaggle/input/food-com-recipes-and-user-interactions/PP_users.csv
/kaggle/input/food-com-recipes-and-user-interactions/interactions_validation.csv


In [3]:
import pandas as pd
from collections import Counter
import ast
import re

# Function to safely evaluate the tags string to a list
def safe_eval_tags(tags_string):
    try:
        # Remove any extra brackets and split the string
        cleaned_string = re.sub(r'\]\[', ', ', tags_string.strip('[]'))
        return ast.literal_eval(f'[{cleaned_string}]')
    except:
        return []

# Simulated function to mimic reading from a dataset with limited rows
def read_limited_csv(file_path, max_rows=10000, chunksize=1000):
    rows_read = 0
    for chunk in pd.read_csv(file_path, chunksize=chunksize):
        if rows_read + len(chunk) > max_rows:
            yield chunk.iloc[:max_rows - rows_read]
            break
        yield chunk
        rows_read += len(chunk)

# Initialize a global tag counter
global_tag_counter = Counter()

# Process the data in chunks
chunk_number = 0
max_rows = 10000
for chunk in read_limited_csv('/kaggle/input/food-com-recipes-and-user-interactions/RAW_recipes.csv', max_rows=max_rows):
    chunk_number += 1

    # Initialize a tag counter for this chunk
    chunk_tag_counter = Counter()

    # Process tags in this chunk
    for tags in chunk['tags']:
        tag_list = safe_eval_tags(tags)
        chunk_tag_counter.update(tag_list)

    # Update global counter
    global_tag_counter.update(chunk_tag_counter)

    # Print chunk-level results
    print(f"\nChunk {chunk_number}:")
    print(f"Total Recipes in Chunk: {len(chunk)}")
    print("Top 5 Tags in Chunk:")
    for tag, count in chunk_tag_counter.most_common(5):
        print(f"{tag}: {count}")

# Get the total number of recipes processed
total_recipes = max_rows

# Print global results
print("\nGlobal Tag Analysis:")
print(f"Total unique tags: {len(global_tag_counter)}")
print("Top 20 tags across all processed rows:")
for tag, count in global_tag_counter.most_common(20):
    percentage = (count / total_recipes) * 100
    print(f"{tag}: {count} ({percentage:.2f}%)")

# Least common tags
print("\nBottom 20 least common tags:")
for tag, count in global_tag_counter.most_common()[-20:]:
    percentage = (count / total_recipes) * 100
    print(f"{tag}: {count} ({percentage:.2f}%)")

# Tags appearing in more than 10% of recipes
common_tags = [tag for tag, count in global_tag_counter.items() if (count / total_recipes) > 0.1]
print(f"\nTags that appear in more than 10% of recipes: {common_tags}")

# Tags appearing in less than 1% of recipes
rare_tags = [tag for tag, count in global_tag_counter.items() if (count / total_recipes) < 0.01]
print(f"\nNumber of tags that appear in less than 1% of recipes: {len(rare_tags)}")



Chunk 1:
Total Recipes in Chunk: 1000
Top 5 Tags in Chunk:
preparation: 993
time-to-make: 965
course: 961
dietary: 711
main-ingredient: 674

Chunk 2:
Total Recipes in Chunk: 1000
Top 5 Tags in Chunk:
preparation: 991
time-to-make: 954
course: 950
dietary: 709
main-ingredient: 676

Chunk 3:
Total Recipes in Chunk: 1000
Top 5 Tags in Chunk:
preparation: 994
time-to-make: 970
course: 931
dietary: 723
main-ingredient: 696

Chunk 4:
Total Recipes in Chunk: 1000
Top 5 Tags in Chunk:
preparation: 998
time-to-make: 981
course: 953
dietary: 692
main-ingredient: 643

Chunk 5:
Total Recipes in Chunk: 1000
Top 5 Tags in Chunk:
preparation: 996
time-to-make: 975
course: 954
main-ingredient: 679
dietary: 671

Chunk 6:
Total Recipes in Chunk: 1000
Top 5 Tags in Chunk:
preparation: 995
time-to-make: 982
course: 934
dietary: 719
main-ingredient: 648

Chunk 7:
Total Recipes in Chunk: 1000
Top 5 Tags in Chunk:
preparation: 999
time-to-make: 983
course: 973
main-ingredient: 856
fruit: 769

Chunk 8:
Total

In [4]:
import re

# Refine the filtering logic to exclude non-time-related tags
def is_time_related(tag):
    # Match tags that explicitly include time units like minutes, hours, or specific patterns like "1-day-or-more"
    time_patterns = [
        r"\b\d+-minutes?\b",  # Matches "15-minutes", "30-minutes", etc.
        r"\b\d+-hours?\b",    # Matches "4-hours", "1-hour", etc.
        r"\b\d+-day(?:s)?-or-more\b",  # Matches "1-day-or-more"
        r"\b\d+-minutes-or-less\b",  # Matches "30-minutes-or-less", "15-minutes-or-less", etc.
        r"\b\d+-hours-or-less\b",    # Matches "4-hours-or-less"
    ]
    return any(re.search(pattern, tag) for pattern in time_patterns)

# Apply the refined logic
time_related_tags = {tag: count for tag, count in global_tag_counter.items() if is_time_related(tag)}

# Print the refined time-related tags
print("Refined Time-related tags:")
for tag, count in sorted(time_related_tags.items(), key=lambda x: x[1], reverse=True):
    print(f"{tag}: {count}")

# Display the top 10 most common refined time-related tags
print("\nTop 10 refined most common time-related tags:")
for tag, count in sorted(time_related_tags.items(), key=lambda x: x[1], reverse=True)[:10]:
    print(f"{tag}: {count}")


Refined Time-related tags:
60-minutes-or-less: 2913
30-minutes-or-less: 2195
4-hours-or-less: 2195
15-minutes-or-less: 2172
1-day-or-more: 113

Top 10 refined most common time-related tags:
60-minutes-or-less: 2913
30-minutes-or-less: 2195
4-hours-or-less: 2195
15-minutes-or-less: 2172
1-day-or-more: 113


In [5]:
# # Categorize tags (this is a manual process, here's a start)
# time_of_day_tags = {'breakfast', 'brunch', 'lunch', 'dinner', 'snack'}
# ingredient_tags = {'fruits', 'vegetables', 'meat', 'poultry', 'seafood', 'dairy'}
# preparation_tags = {'easy', 'quick', '15-minutes-or-less', '30-minutes-or-less', '60-minutes-or-less'}
# occasion_tags = {'christmas', 'thanksgiving', 'halloween', 'easter', 'holiday', 'summer', 'winter', 'fall', 'spring'}

# # Function to check if a recipe has tags from a specific category
# def has_tag_from_category(tags, category_tags):
#     return bool(set(eval(tags)) & category_tags)

# # Add boolean columns for each category
# df['is_breakfast'] = df['tags'].apply(lambda x: has_tag_from_category(x, {'breakfast'}))
# df['is_lunch'] = df['tags'].apply(lambda x: has_tag_from_category(x, {'lunch'}))
# df['is_dinner'] = df['tags'].apply(lambda x: has_tag_from_category(x, {'dinner'}))
# df['is_easy'] = df['tags'].apply(lambda x: has_tag_from_category(x, {'easy'}))
# df['is_quick'] = df['tags'].apply(lambda x: has_tag_from_category(x, preparation_tags))
# df['is_holiday'] = df['tags'].apply(lambda x: has_tag_from_category(x, occasion_tags))
# df['has_fruits'] = df['tags'].apply(lambda x: has_tag_from_category(x, {'fruits'}))

# print(df[['name', 'is_breakfast', 'is_lunch', 'is_dinner', 'is_easy', 'is_quick', 'is_holiday', 'has_fruits']].head())

## Aim is to utilize the tags for content based similarity based on tags maybe levraging a little bit of NLP towards the end
- first goal : get results on RAW data
- second goal : pre-process data group them into more better / informative tags

In [6]:
import pandas as pd
import ast
import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# Define a function to extract time-related tags
def extract_time_tags(tags):
    time_patterns = [
        r"\b\d+-minutes?\b",         # Matches "15-minutes", "30-minutes", etc.
        r"\b\d+-hours?\b",           # Matches "4-hours", "1-hour", etc.
        r"\b\d+-day(?:s)?-or-more\b",  # Matches "1-day-or-more"
        r"\b\d+-minutes-or-less\b",  # Matches "30-minutes-or-less", "15-minutes-or-less", etc.
        r"\b\d+-hours-or-less\b"     # Matches "4-hours-or-less"
    ]
    try:
        # Convert string to list
        tag_list = ast.literal_eval(tags)
        # Filter tags matching the time patterns
        return [tag for tag in tag_list if any(re.search(pattern, tag) for pattern in time_patterns)]
    except:
        return []

# Read a sample of the dataset
file_path = '/kaggle/input/food-com-recipes-and-user-interactions/RAW_recipes.csv'
chunk_size = 5000  # Define chunk size for processing
sample_frac = 0.1  # Fraction of data to sample

# Process the dataset in chunks to sample
dfs = []
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    sampled_chunk = chunk.sample(frac=sample_frac, random_state=42)
    dfs.append(sampled_chunk)

# Combine sampled chunks
recipes_df = pd.concat(dfs)

# Extract time-related tags into a new column
recipes_df['time_tags'] = recipes_df['tags'].apply(extract_time_tags)

# Drop rows with no time tags
recipes_df = recipes_df[recipes_df['time_tags'].apply(len) > 0]

# Convert time_tags to a string format for vectorization
recipes_df['time_tags_str'] = recipes_df['time_tags'].apply(lambda tags: ' '.join(tags))

# Use CountVectorizer to create a binary matrix for the time tags
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(' '))
time_tags_matrix = vectorizer.fit_transform(recipes_df['time_tags_str'])

# Compute cosine similarity on the matrix
similarity_matrix = cosine_similarity(time_tags_matrix)

# Convert the similarity matrix to a DataFrame for easy lookup
similarity_df = pd.DataFrame(similarity_matrix, index=recipes_df.index, columns=recipes_df.index)

# Example: Find top 3 similar recipes for a given recipe index
def get_similar_recipes(recipe_index, top_n=3):
    # Sort recipes by similarity score
    similar_scores = similarity_df[recipe_index].sort_values(ascending=False)
    # Exclude the recipe itself and return top N recommendations
    similar_recipes = similar_scores.drop(recipe_index).head(top_n)
    return similar_recipes

# Test the recommendation system with a sample recipe
recipe_index = recipes_df.index[0]  # Select the first recipe index
recommended_recipes = get_similar_recipes(recipe_index, top_n=3)

# Print recommendations
print(f"Top {len(recommended_recipes)} similar recipes for Recipe {recipe_index} based on time tags:")
print(recommended_recipes)

# Save the filtered and sampled dataframe for future use
recipes_df.to_csv('sampled_recipes_with_time_tags.csv', index=False)


/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Top 3 similar recipes for Recipe 1501 based on time tags:
110177    1.0
114392    1.0
112509    1.0
Name: 1501, dtype: float64


In [7]:
# Access specific rows by index
rows_of_interest = recipes_df.loc[[110177, 114392, 112509]]
rows_of_interest


,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,time_tags,time_tags_str
110177,i can t wait for chole,25632,35,37636,2002-04-18,"['60-minutes-or-less', 'time-to-make', 'course...","[300.0, 4.0, 16.0, 26.0, 24.0, 1.0, 19.0]",11,"['in large saucepan over medium heat , heat cu...","a nice spicy indian chickpea recipe. i know, i...","['onions', 'fresh ginger', 'garlic', 'cumin se...",14,[60-minutes-or-less],60-minutes-or-less
114392,japanese crispy fried chicken kara age,494787,40,2678032,2013-02-05,"['lactose', '60-minutes-or-less', 'time-to-mak...","[397.4, 26.0, 6.0, 35.0, 47.0, 24.0, 11.0]",7,"['cut chicken in bite size pieces', 'in a plas...",this is a classic japanese kara-age recipe. ch...,"['boneless chicken thighs', 'ginger', 'clove',...",8,[60-minutes-or-less],60-minutes-or-less
112509,italian christmas cookies filled,420915,60,1559190,2010-04-19,"['60-minutes-or-less', 'time-to-make', 'course...","[200.6, 13.0, 39.0, 0.0, 4.0, 9.0, 9.0]",12,"['sift together flours and salt', 'cut shorten...",these are cookies my grandmother used to make ...,"['pastry flour', 'flour', 'salt', 'shortening'...",10,[60-minutes-or-less],60-minutes-or-less


In [8]:
# recipes_df.iloc[0]

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ast

# Function to tokenize ingredients
def tokenize_ingredients(ingredients):
    # Flatten ingredient list into a single string
    ingredient_str = ' '.join(ingredients).lower()
    return ingredient_str.split()  # Tokenize into words (no stopword removal)

# Apply tokenization to the ingredients column
recipes_df['processed_ingredients'] = recipes_df['ingredients'].apply(
    lambda x: tokenize_ingredients(ast.literal_eval(x))
)

# Combine time_tags and processed_ingredients
def combine_features(row):
    time_tags_str = ' '.join(row['time_tags'])
    ingredients_str = ' '.join(row['processed_ingredients'])
    return f"{time_tags_str} {ingredients_str}"

recipes_df['combined_features'] = recipes_df.apply(combine_features, axis=1)

# Use CountVectorizer to vectorize the combined_features column
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(' '))
combined_features_matrix = vectorizer.fit_transform(recipes_df['combined_features'])

# Compute cosine similarity on the combined feature matrix
combined_similarity_matrix = cosine_similarity(combined_features_matrix)

# Convert the similarity matrix to a DataFrame for easy lookup
combined_similarity_df = pd.DataFrame(combined_similarity_matrix, index=recipes_df.index, columns=recipes_df.index)

# Function to get top N similar recipes
def get_similar_recipes(recipe_index, top_n=25):
    # Sort recipes by similarity score
    similar_scores = combined_similarity_df[recipe_index].sort_values(ascending=False)
    # Exclude the recipe itself and return top N recommendations
    similar_recipes = similar_scores.drop(recipe_index).head(top_n)
    return similar_recipes

# Test the recommendation system with a sample recipe
recipe_index = recipes_df.index[0]  # Replace with your recipe ID
recommended_recipes = get_similar_recipes(recipe_index, top_n=25)

# Print recommendations
print(f"Top {len(recommended_recipes)} similar recipes for Recipe {recipes_df.iloc[recipe_index]['name']} (Index: {recipe_index}):")
print(recommended_recipes)

# Access details of recommended recipes
print("\nDetails of recommended recipes:")
print(recipes_df.loc[recommended_recipes.index, ['name', 'ingredients', 'time_tags']])



/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Top 25 similar recipes for Recipe baked shrimp toasts (Index: 1501):
13907     0.782624
139758    0.737865
173899    0.717137
123625    0.711512
128456    0.700000
94923     0.700000
226020    0.700000
28106     0.700000
110691    0.676123
28957     0.676123
151907    0.676123
143181    0.670820
187417    0.667424
48673     0.667424
52225     0.667424
22232     0.667424
102907    0.667424
67483     0.667424
69415     0.667424
212594    0.667424
131973    0.653197
22413     0.653197
199771    0.652929
201647    0.652929
220468    0.652929
Name: 1501, dtype: float64

Details of recommended recipes:
                                                   name  \
13907                 baked cinnamon sugar french toast   
139758          mother s one hour rolls  mors tvebakker   
173899                               rich baked custard   
123625                               light brioche buns   
128456                                        magic pie   
94923                        grandma chin 

In [10]:
recipes_df['combined_features']

1501      60-minutes-or-less dry yeast milk butter eggs ...
2586      60-minutes-or-less cooked chicken breast dry m...
2653      4-hours-or-less all-purpose flour whole wheat ...
1055      15-minutes-or-less desiccated coconut sweetene...
705       15-minutes-or-less cranberry juice cocktail un...
                                ...                        
231022    4-hours-or-less eggs unsweetened applesauce va...
230168    15-minutes-or-less light chunk tuna in water b...
230680    60-minutes-or-less tilapia fillets fresh coria...
231182    60-minutes-or-less butter zucchini fresh tarra...
230602    60-minutes-or-less hot pepper sauce green pepp...
Name: combined_features, Length: 22021, dtype: object

### Let's remove Time - Tags and see what the next most occuring domain

In [26]:
from collections import Counter
import ast

# Flatten all tags into a single list
all_tags = []
recipes_df['tags'].apply(lambda x: all_tags.extend(ast.literal_eval(x)))

# Count the occurrences of each tag
tag_counter = Counter(all_tags)

# Sort tags by frequency
sorted_tags = tag_counter.most_common()

# Filter out time-related tags
filtered_tags = [(tag, count) for tag, count in sorted_tags if tag not in time_related_tags]

# # Display the filtered tags descending order
# print("Top 200 most common tags (excluding time-related tags):")
# for tag, count in filtered_tags[:200]:
#     print(f"{tag}: {count}")

# # Display the top 100 filtered tags
# print("Top 100 most common tags (excluding time-related tags):")
# for tag, count in filtered_tags[:100]:
#     print(f"{tag}: {count}")

# Total unique tags after filtering
print(f"\nTotal unique tags (excluding time-related tags): {len(filtered_tags)}")




Total unique tags (excluding time-related tags): 480


In [28]:
import ast

# Define tag categories
time_tags = ['60-minutes-or-less','30-minutes-or-less','4-hours-or-less','15-minutes-or-less','1-day-or-more']
course_tags = ['main-dish', 'side-dishes', 'desserts', 'appetizers', 'salads', 'snacks', 'soups-stews', 'breakfast', 'lunch', 'dinner-party', 'brunch']
cuisine_tags = ['north-american', 'european', 'mexican', 'italian', 'asian', 'middle-eastern', 'greek', 'southern-united-states', 'indian', 'caribbean', 'african']
dietary_tags = ['vegetarian', 'vegan', 'low-carb', 'low-fat', 'low-sodium', 'gluten-free', 'low-calorie', 'diabetic', 'low-cholesterol']
ingredient_tags = ['vegetables', 'meat', 'seafood', 'fruit', 'eggs-dairy', 'pasta', 'poultry', 'beef', 'pork', 'nuts', 'cheese']
taste_mood_tags = ['sweet', 'savory', 'spicy', 'comfort-food', 'romantic', 'healthy', 'kid-friendly', 'beginner-cook']
occasion_tags = ['holiday-event', 'christmas', 'thanksgiving', 'valentines-day', 'summer', 'fall', 'winter', 'spring', 'picnic', 'to-go']
technique_tags = ['oven', 'stove-top', 'grilling', 'crock-pot-slow-cooker', 'broil', 'baking', 'stir-fry']
equipment_tags = ['food-processor-blender', 'small-appliance', 'refrigerator', 'microwave']
health_tags = ['low-saturated-fat', 'high-calcium', 'high-protein', 'very-low-carbs', 'low-sugar']
specific_ingredients_tags = ['chocolate', 'berries', 'tropical-fruit', 'onions', 'citrus', 'potatoes', 'carrots', 'mushrooms']

# Function to filter tags into categories
def categorize_tags(tags, category_list):
    tags = ast.literal_eval(tags)  # Convert string representation of list to actual list
    return [tag for tag in tags if tag in category_list]

# Create new columns for each category
recipes_df['time_tags'] = recipes_df['tags'].apply(lambda x: categorize_tags(x, time_tags))
recipes_df['course_tags'] = recipes_df['tags'].apply(lambda x: categorize_tags(x, course_tags))
recipes_df['cuisine_tags'] = recipes_df['tags'].apply(lambda x: categorize_tags(x, cuisine_tags))
recipes_df['dietary_tags'] = recipes_df['tags'].apply(lambda x: categorize_tags(x, dietary_tags))
recipes_df['ingredient_tags'] = recipes_df['tags'].apply(lambda x: categorize_tags(x, ingredient_tags))
recipes_df['taste_mood_tags'] = recipes_df['tags'].apply(lambda x: categorize_tags(x, taste_mood_tags))
recipes_df['occasion_tags'] = recipes_df['tags'].apply(lambda x: categorize_tags(x, occasion_tags))
recipes_df['technique_tags'] = recipes_df['tags'].apply(lambda x: categorize_tags(x, technique_tags))
recipes_df['equipment_tags'] = recipes_df['tags'].apply(lambda x: categorize_tags(x, equipment_tags))
recipes_df['health_tags'] = recipes_df['tags'].apply(lambda x: categorize_tags(x, health_tags))
recipes_df['specific_ingredients_tags'] = recipes_df['tags'].apply(lambda x: categorize_tags(x, specific_ingredients_tags))

# Verify results
recipes_df[['tags', 'course_tags', 'cuisine_tags', 'dietary_tags']].head()


,tags,course_tags,cuisine_tags,dietary_tags
1501,"['60-minutes-or-less', 'time-to-make', 'course', 'cuisine', 'preparation', 'occasion', 'north-american', 'pancakes-and-waffles', 'breakfast', 'american', 'dietary', 'copycat', 'midwestern', 'novelty']",[breakfast],[north-american],[]
2586,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'main-dish', 'beans', 'poultry', 'mexican', 'easy', 'chicken', 'one-dish-meal', 'inexpensive', 'black-beans', 'meat', 'chicken-breasts', 'novelty']",[main-dish],"[north-american, mexican]",[]
2653,"['danish', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'healthy', 'pancakes-and-waffles', 'breakfast', 'eggs-dairy', 'scandinavian', 'european', 'kid-friendly', 'stove-top', 'dutch', 'dietary', 'low-sodium', 'inexpensive', 'toddler-friendly', 'low-in-something', 'brunch', 'equipment', '4-hours-or-less']","[breakfast, brunch]",[european],[low-sodium]
1055,"['15-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', '5-ingredients-or-less', 'desserts', 'fruit', 'easy', 'cookies-and-brownies', 'nuts', 'dietary', 'high-calcium', 'high-in-something', 'coconut']",[desserts],[],[]
705,"['15-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'for-large-groups', 'low-protein', 'healthy', 'beverages', 'fruit', 'easy', 'refrigerator', 'dinner-party', 'holiday-event', 'low-fat', 'cocktails', 'punch', 'nuts', 'dietary', 'gifts', 'spicy', 'christmas', 'low-sodium', 'low-cholesterol', 'low-saturated-fat', 'low-calorie', 'comfort-food', 'valentines-day', 'healthy-2', 'low-in-something', 'berries', 'citrus', 'oranges', 'coconut', 'tropical-fruit', 'pineapple', 'brunch', 'mardi-gras-carnival', 'taste-mood', 'sweet', 'equipment', 'number-of-servings', 'presentation', 'served-cold']","[dinner-party, brunch]",[],"[low-fat, low-sodium, low-cholesterol, low-calorie]"


In [30]:
# A one stop code for Recommendation based on all tags.
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ast

# Function to combine selected tags and features into a single text string
def combine_selected_features(row, tag_columns):
    combined_str = ''
    for col in tag_columns:
        if col in row:
            combined_str += ' '.join(row[col]) + ' '
    return combined_str.strip()

# Define the tag categories you want to use
tag_columns_to_use = [
    'time_tags', 'course_tags', 'cuisine_tags', 'dietary_tags',
    'ingredient_tags', 'taste_mood_tags', 'occasion_tags',
    'technique_tags', 'equipment_tags', 'health_tags', 'specific_ingredients_tags'
]

# Combine selected tags into a single feature column
recipes_df['selected_features'] = recipes_df.apply(
    lambda row: combine_selected_features(row, tag_columns_to_use), axis=1
)

# Use CountVectorizer to vectorize the combined features
vectorizer = CountVectorizer(tokenizer=lambda x: x.split(' '))
selected_features_matrix = vectorizer.fit_transform(recipes_df['selected_features'])

# Compute cosine similarity
similarity_matrix = cosine_similarity(selected_features_matrix)

# Convert similarity matrix to a DataFrame for easier lookup
similarity_df = pd.DataFrame(similarity_matrix, index=recipes_df.index, columns=recipes_df.index)

# Function to get top N similar recipes based on combined tags
def get_similar_recipes(recipe_index, top_n=25):
    # Sort recipes by similarity score
    similar_scores = similarity_df[recipe_index].sort_values(ascending=False)
    # Exclude the recipe itself and return top N recommendations
    similar_recipes = similar_scores.drop(recipe_index).head(top_n)
    return similar_recipes

# Test the recommendation system with a sample recipe
sample_recipe_index = recipes_df.index[0]  # Replace with a specific recipe ID if needed
recommended_recipes = get_similar_recipes(sample_recipe_index, top_n=25)

# Print recommendations
print(f"Top {len(recommended_recipes)} similar recipes for Recipe {recipes_df.iloc[sample_recipe_index]['name']} (Index: {sample_recipe_index}):")
print(recommended_recipes)

# Access details of recommended recipes
print("\nDetails of recommended recipes:")
print(recipes_df.loc[recommended_recipes.index, ['name', 'selected_features']])


/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Top 25 similar recipes for Recipe baked shrimp toasts (Index: 1501):
50084     1.000000
199749    0.866025
151293    0.816497
23347     0.816497
122107    0.816497
130393    0.816497
23064     0.816497
136183    0.816497
226741    0.816497
15221     0.816497
150626    0.816497
155691    0.816497
28504     0.816497
166669    0.816497
167483    0.816497
168825    0.816497
14966     0.816497
201647    0.816497
208135    0.816497
211907    0.816497
102623    0.816497
21600     0.816497
103295    0.816497
27758     0.816497
52164     0.816497
Name: 1501, dtype: float64

Details of recommended recipes:
                                                      name  \
50084                       chocolate chip sundrop muffins   
199749   sticky bun breakfast ring with cream cheese icing   
151293     overnight cinnamon swirl cranberry french toast   
23347                                   big apple pancakes   
122107                            lemon pound cake muffins   
130393                  

In [32]:
# A Genre Type View Of Recommendation

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import ast

# Function to combine features for a specific group of tags
def combine_group_features(row, tag_column):
    if tag_column in row and isinstance(row[tag_column], list):
        return ' '.join(row[tag_column])
    return ''

# Function to calculate and return recommendations for a specific group
def get_recommendations_for_group(recipes_df, tag_column, recipe_index, top_n=25):
    # Combine features for the specified tag column
    recipes_df[f'{tag_column}_features'] = recipes_df.apply(
        lambda row: combine_group_features(row, tag_column), axis=1
    )
    
    # Vectorize the group-specific features
    vectorizer = CountVectorizer(tokenizer=lambda x: x.split(' '))
    features_matrix = vectorizer.fit_transform(recipes_df[f'{tag_column}_features'])
    
    # Compute cosine similarity
    similarity_matrix = cosine_similarity(features_matrix)
    
    # Convert similarity matrix to DataFrame
    similarity_df = pd.DataFrame(similarity_matrix, index=recipes_df.index, columns=recipes_df.index)
    
    # Get top N similar recipes
    similar_scores = similarity_df[recipe_index].sort_values(ascending=False)
    similar_recipes = similar_scores.drop(recipe_index).head(top_n)
    return similar_recipes

# Example: Generate recommendations for each group of tags
tag_groups = [
    'time_tags', 'course_tags', 'cuisine_tags', 'dietary_tags',
    'ingredient_tags', 'taste_mood_tags', 'occasion_tags',
    'technique_tags', 'equipment_tags', 'health_tags', 'specific_ingredients_tags'
]

# Recipe index to test
sample_recipe_index = recipes_df.index[0]  # Replace with your recipe index
all_recommendations = {}

for group in tag_groups:
    try:
        recommendations = get_recommendations_for_group(
            recipes_df, group, sample_recipe_index, top_n=10
        )
        all_recommendations[group] = recommendations
        print(f"\nRecommendations based on {group}:")
        print(recipes_df.loc[recommendations.index, ['name', group]])
    except ValueError as e:
        print(f"\nNo valid features for {group}: {e}")

# Access all recommendations
print("\nSummary of Recommendations:")
for group, recs in all_recommendations.items():
    print(f"\n{group} Recommendations:")
    print(recipes_df.loc[recs.index, ['name']])


/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(



Recommendations based on time_tags:
                                            name             time_tags
110177                    i can t wait for chole  [60-minutes-or-less]
114392  japanese crispy fried chicken   kara age  [60-minutes-or-less]
112509         italian christmas cookies  filled  [60-minutes-or-less]
111323         individual sweet potato crostatas  [60-minutes-or-less]
111242           individual breakfast casseroles  [60-minutes-or-less]
112450        italian chicken and vegetable soup  [60-minutes-or-less]
112753                   italian mashed potatoes  [60-minutes-or-less]
113197                         italian struffoli  [60-minutes-or-less]
112623           italian garlic roasted potatoes  [60-minutes-or-less]
114995                                jerk wings  [60-minutes-or-less]


/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(



Recommendations based on course_tags:
                                                     name  course_tags
53574                           cinnamon breakfast apples  [breakfast]
220676                            vanilla ricotta muffins  [breakfast]
28673                             breakfast tofu smoothie  [breakfast]
57586   copy cat   cb hash brown casserole by todd wilbur  [breakfast]
165227                potatoes  with everything  pancakes  [breakfast]
222754                          venison breakfast sausage  [breakfast]
84686                                    flamenco eggs ii  [breakfast]
53957                       cinnamon rolls    kanelbullar  [breakfast]
28429                               breakfast burrito  ww  [breakfast]
202778                     sugar hill inn s raisin scones  [breakfast]


/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


...


Recommendations based on equipment_tags:
                                                                name  \
154838                                                payday cookies   
154076  pasta with pancetta  broccoli or broccoli rabe and pine nuts   
152483                                    pap  potatoes and cornmeal   
153653                                               pasta e fagioli   
150999                              oven roasted santa maria tri tip   
153145               parmesan crusted chicken on bed of fancy greens   
150911                                      oven meatballs and gravy   
153979                         pasta with creamy bacon and pea sauce   
153425                                     passion fruit vinaigrette   
152357                                                 panfried fish   

       equipment_tags  
154838             []  
154076             []  
152483             []  
153653             []  
150999             []  
153145             []

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(



Recommendations based on health_tags:
                                                        name health_tags
147377                                    ole  cheese spread          []
148682                                 orange blossom sorbet          []
147215  old fashioned banana cream pie with chocolate pastry          []
148340      oodles of noodles   butter poppy seed  variation          []
147633                                 olives  italian style          []
148862                                    orange cream candy          []
149257                      orange poppy seed muffins  light          []
149395                                        orange slushie          []
145168                                   norwegian gold cake          []
140367                                             mujadarra          []


/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(



Recommendations based on specific_ingredients_tags:
                                                  name  \
147323                      old fashioned rice pudding   
149816                           oregano dijon chicken   
146057        oatmeal and bananas breakfast  baby food   
147351        old style mustard and rosemary asparagus   
145240                               not chopped liver   
147094                         old fashioned tea cakes   
147123  old mexican inn dip  serve with tortilla chips   
148601                       orange and lemon tea fizz   
149209                                 orange pancakes   
145079                            north woods pancakes   

       specific_ingredients_tags  
147323                        []  
149816                        []  
146057                        []  
147351                        []  
145240                        []  
147094                        []  
147123                        []  
148601                      